In [ ]:
# File: codim.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
from frontend import App

# create an app
app = App.create("codim")

# create an armadillo tetrahedral mesh
V, F, T = app.mesh.preset("armadillo").decimate(30000).tetrahedralize().normalize()
app.asset.add.tet("armadillo", V, F, T)

# create a 1D rod strand mesh
V, E = app.mesh.line([0, 0, -1], [0, 0, 1], 64)
app.asset.add.rod("strand", V, E)

# create a scene
scene = app.scene.create()

# stack multiple armadillos vertically with spacing
space = 0.9
for k in range(3):
    armadillo = scene.add("armadillo").scale(0.75).rotate(180, "y")
    armadillo.at(0, 0.75 + space * k, 0).jitter()
    armadillo.param.set("young-mod", 2000)

# add a curtain of vertical strands
N = 21
for i in range(N):
    x = (i - N // 2) / (N // 2)
    strand = scene.add("strand").at(x, 0, 0)
    strand.param.set("young-mod", 1e7).set("friction", 0.5)
    # pin top and bottom of each strand
    strand.pin(strand.grab([0, 0, -1]) + strand.grab([0, 0, 1]))

# add invisible walls to contain the simulation
gap = 0.025
scene.add.invisible.wall([1 + gap, 0, 0], [-1, 0, 0])
scene.add.invisible.wall([-1 - gap, 0, 0], [1, 0, 0])
scene.add.invisible.wall([0, 0, 1 + gap], [0, 0, -1])
scene.add.invisible.wall([0, 0, -1 - gap], [0, 0, 1])

# set preview options
opts = {"lookat": [0, 0.8, 0], "eyeup": 0.1, "fov": 50}

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview(options=opts)

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters
session.param.set("frames", 240)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview(options=opts)

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate(options=opts)

In [ ]:
# export the animation to file and clear temp files
session.export.animation(clear=True)

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()